In [1]:
# Installation des packages
!pip install -q torch transformers sentence-transformers langchain chromadb pypdf langdetect accelerate bitsandbytes
!pip install -q unstructured pdf2image
!pip install -U langchain-community
!pip install -q flask flask-cors pyngrok

print("✅ Toutes les dépendances sont installées")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import files
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langdetect import detect
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import gc
import os

# Configuration
MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
NGROK_AUTH_TOKEN = "votre_token_ngrok"  # À remplacer

print("✅ Configuration et imports terminés")

✅ Configuration et imports terminés


In [3]:
# Fonctions de traitement de documents
def upload_pdfs():
    uploaded = files.upload()
    return [f for f in uploaded if f.lower().endswith('.pdf')]

def process_documents(pdf_files):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    documents = []
    for pdf in pdf_files:
        loader = PyPDFLoader(pdf)
        documents.extend(loader.load())
    return text_splitter.split_documents(documents)

def create_vector_store(texts):
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    return Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory="rdsp_db"
    )

print("✅ Fonctions documentaires prêtes")

✅ Fonctions documentaires prêtes


In [4]:
def load_zephyr_model():
    gc.collect()
    torch.cuda.empty_cache()

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # Solution 1: Essai avec CUDA si disponible
    if torch.cuda.is_available():
        try:
            model = AutoModelForCausalLM.from_pretrained(
                MODEL_NAME,
                device_map="auto",
                torch_dtype=torch.float16
            )
            print("✅ Modèle chargé avec CUDA (16-bit)")
            return pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512,
                temperature=0.7
            )
        except Exception as e:
            print(f"⚠ Erreur CUDA: {e}\nTentative CPU...")

    # Solution 2: Fallback CPU optimisé
    try:
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float32,
            device_map="cpu",
            low_cpu_mem_usage=True
        )
        print("✅ Modèle chargé sur CPU (mémoire optimisée)")
        return pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,  # Réduit pour CPU
            temperature=0.7
        )
    except Exception as e:
        print(f"❌ Erreur critique: {e}")
        print("Solutions alternatives:")
        print("1. Changez de runtime Colab (Menu > Runtime > Changer le type de runtime)")
        print("2. Essayez un modèle plus petit comme 'HuggingFaceH4/zephyr-3b-beta'")
        raise

print("⚠️ Chargement du modèle (patientez 3-10 minutes)...")
try:
    zephyr_pipe = load_zephyr_model()
    print("✅ Prêt pour l'inférence!")
except Exception as e:
    print(f"Échec du chargement: {e}")

⚠️ Chargement du modèle (patientez 3-10 minutes)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Modèle chargé avec CUDA (16-bit)
✅ Prêt pour l'inférence!


In [5]:
class RDSPChatbot:
    def __init__(self):
        self.user_profile = {}
        self.conversation_history = []

    def _calculate_benefits(self, income):
        """Calcule les avantages selon le revenu (chiffres 2023)"""
        if income <= 35000:
            return {'max_contribution': 1500, 'grant': 3500, 'bond': 1000}
        elif income <= 45000:
            return {'max_contribution': 1000, 'grant': 2000, 'bond': 500}
        else:
            return {'max_contribution': 500, 'grant': 1000, 'bond': 0}

    def _extract_user_data(self, question):
        """Extrait revenu/âge de la question"""
        # Détection revenu
        if any(kw in question.lower() for kw in ["revenu", "gagne", "salaire"]):
            try:
                self.user_profile['income'] = int(''.join(filter(str.isdigit,
                                               question.split("cad")[0].split()[-1])))
            except:
                pass

        # Détection âge
        if any(kw in question.lower() for kw in ["âge", "ans", "vie"]):
            try:
                self.user_profile['age'] = int(''.join(filter(str.isdigit,
                                            question.split("ans")[0].split()[-1])))
            except:
                pass

    def ask(self, question, vector_store):
        # 1. Extraction données utilisateur
        self._extract_user_data(question)

        # 2. Réponses personnalisées
        if "cotis" in question.lower() and 'income' in self.user_profile:
            benefits = self._calculate_benefits(self.user_profile['income'])
            return (
                f"Pour {self.user_profile['income']}$ CAD/an :\n"
                f"• Votre cotisation optimale : {benefits['max_contribution']}$\n"
                f"• Subvention gouvernementale : {benefits['grant']}$\n"
                f"• Bon canadien : {benefits['bond']}$\n"
                f"Total annuel potential : {sum(benefits.values())}$"
            )

        if "âge" in question.lower() and 'age' in self.user_profile:
            eligible = "éligible" if self.user_profile['age'] < 60 else "non éligible"
            return f"À {self.user_profile['age']} ans : {eligible} (limite : 59 ans)"

        # 3. Recherche documentaire standard
        docs = vector_store.similarity_search(question, k=1)
        context = docs[0].page_content if docs else ""

        # 4. Génération de réponse contrôlée
        prompt = f"""<|system|>
        [Règles]
        - Réponse exclusive sur le RDSP canadien
        - Maximum 2 phrases
        - Structure : "[Réponse] : [information]"
        - Contexte : {context}
        </s>
        <|user|>{question}</s>
        <|assistant|>RÉPONSE :"""

        response = zephyr_pipe(
            prompt,
            max_new_tokens=100,
            temperature=0.1,
            do_sample=False
        )[0]['generated_text'].split("RÉPONSE :")[-1].strip()

        # 5. Validation finale
        if not response or "RDSP" not in response.upper():
            return "Je ne peux répondre qu'aux questions sur le RDSP canadien."

        self.conversation_history.append((question, response))
        return response

# Initialisation
chatbot = RDSPChatbot()

In [6]:
# 1. Upload des PDFs
print("📤 Veuillez uploader vos fichiers PDF")
pdf_files = upload_pdfs()

# 2. Traitement des documents
print("🔄 Traitement des documents...")
texts = process_documents(pdf_files)
vector_store = create_vector_store(texts)

# 3. Initialisation du chatbot amélioré
chatbot = RDSPChatbot()

# 4. Test rapide (optionnel)
print("\n🔍 Test rapide du chatbot amélioré")
test_q = "Je gagne 30000 CAD par an, que me conseillez-vous ?"
print(f"Q: {test_q}\nR: {chatbot.ask(test_q, vector_store)}")

# 5. Configuration de l'API Flask (PARTIE MODIFIÉE)
def run_flask_app():
    app = Flask(__name__)
    CORS(app)

    # --- ROUTE MODIFIÉE ICI ---
    @app.route('/api/chat', methods=['POST'])
    def api_chat():
        data = request.get_json()
        response = chatbot.ask(data['question'], vector_store)  # vector_store ajouté
        return jsonify({"response": response})
    # -------------------------

    app.run(host='0.0.0.0', port=5000)

# 6. Démarrer Ngrok et Flask
start_api = input("\nDémarrer l'API Flask ? (o/n): ")
if start_api.lower() == 'o':
    ngrok.set_auth_token("34GCiX3fgJNlCGMRkPclfoWLpZg_6eXYzqWu1c8syBN6Rrf7k")  # Remplacez par votre token

    # Démarrer Flask dans un thread séparé
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.start()

    # Configurer Ngrok
    public_url = ngrok.connect(5000).public_url
    print(f"\n🌐 API disponible sur : {public_url}/api/chat")
    print("⚠️ Gardez cette cellule active pour maintenir l'API en ligne")

📤 Veuillez uploader vos fichiers PDF


Saving reei.pdf to reei.pdf
🔄 Traitement des documents...


/tmp/ipython-input-3006024413.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔍 Test rapide du chatbot amélioré
Q: Je gagne 30000 CAD par an, que me conseillez-vous ?
R: Si vous souhaitez maximiser vos économies, nous vous recommandons d'explorer les avantages du RDSP canadien. En tant que beneficiaire d'un régime, vous pouvez bénéficier d'une subvention maximale de 3 500 $ par an, jusqu'à l'âge de 49 ans inclus. Recevez un Énoncé d'admissibilité

Démarrer l'API Flask ? (o/n): o
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



🌐 API disponible sur : https://premonarchial-chignoned-hae.ngrok-free.dev/api/chat
⚠️ Gardez cette cellule active pour maintenir l'API en ligne
